In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

In [ ]:
# とりあえずデータ読み込め！
# 選挙結果のデータ
pres08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/pres08.csv')
# 世論調査のデータ
polls08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/polls08.csv')
# それぞれのデータ内に、オバマとマケインの得票率(もしくは支持率)の差を「margin」カラムとして保存
pres08['margin'] = pres08['Obama'] - pres08['McCain']
polls08['margin'] = polls08['Obama'] - polls08['McCain']

# 同じ州の同じ中間日に世論調査が行われているのか？を確認
polls08.groupby(['state','middate']).count()
# 行われてるので、平均をとる必要がある

# pythonでのDate型ってこうやるらしい
polls08['middate'] = pd.to_datetime(polls08['middate'])
# middate は最大でも20081101
polls08['middate'].max()

# 試しに色々計算させてみる
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11')
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16')
import math
abs(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16'))
type(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11'))

# 選挙日までの日数をカラムとして保存
polls08['DaysToElection'] = pd.to_datetime('2008-11-04') - polls08['middate']

# DaysToElectionを小さい方から並べたランクナンバーを振る
polls08 = polls08.assign(
    rankMin=polls08.groupby(by='state')['DaysToElection'].rank(method='min')
)
# 最も選挙日に近い世論調査に限定して、オバマとマケインの差の平均をとり、それを予測値カラム(カラム名：pred)として保存
poll = polls08[polls08['rankMin']==1].groupby(by='state', as_index=False).mean()[['state','margin']]
poll = poll.rename(columns={'margin':'pred'})
# 州の名前をつけ直す
poll = pd.merge(poll, pres08, how='left', on='state')
# 誤差カラムを追加
poll['error'] = poll['margin'] - poll['pred']
# 誤差の平均を算出(＝平均予測誤差)
poll['error'].mean()
# root mean squre(2乗平均平方根)で誤差を見てみる
math.sqrt((poll['error']**2).mean())

In [ ]:
myfig, ax1 =plt.subplots(ncols=1, nrows=1, figsize=(8,6))
ax1 = sns.distplot(poll['error'], ax=ax1, bins=[-20, -15, -10, -5, 0, 5, 10, 15, 20])
ax1.axvline(poll['error'].mean())
ax1.text(x=poll['error'].mean(), y=0.06, s='平均誤差')
myfig.show()

世論調査結果と、実際の選挙結果の関係性をビジュアルで把握するため、x軸に世論調査結果、y軸に選挙結果をプロットする

In [ ]:
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
# 一旦グラフを描かせるために、アルファゼロでプロットしちゃう
ax1 = sns.scatterplot(x='pred', y='margin', data=poll, alpha=0)
# プロットの位置に、州名をおいていく
for index, row in poll.iterrows():
    print(row['pred'], row['margin'], row['state'])
    ax1.text(x=row['pred'], y=row['margin'], s=row['state'], ha='center', va='center')
# 45度線を描く
ax1 = sns.lineplot(x=[-40, 80], y=[-40, 80], ax=ax1, color='gray')
ax1.lines[0].set_linestyle('--')
ax1.axvline(0, linestyle="-", color='gray')
ax1.axhline(0, color='gray')
myfig.show()

今回の選挙結果予測においては、各候補者の支持率差分を選挙結果(得票率)の予測値として使用したが、細かな値の差はさほど重要ではなく、大事なのは、「オバマ勝利と予測して、実際本当にオバマが勝利したかどうか？」といった、「勝った」「負けた」の判断を正しく予測できたか？になる。
そこで、今回の世論調査が、選挙結果を予測するものとして間違っていた州を把握してみる。

In [ ]:
# 予測値、結果、ともに、正負の符号が一致していれば、勝敗の予測が正しかったことになるので、
# 符号が異なっていた州名をリストアップしてみる
poll[(poll['pred'] > 0) != (poll['margin'] > 0)]

どの州も、支持率の差が1％ポイント程度しかない接戦だったことがわかる。